In [1]:
import keepa
import numpy as np
import requests
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from sklearn import linear_model
import scipy.stats as stats
import itertools
from multiprocessing import Pool
import json
from bs4 import BeautifulSoup

In [2]:
with open("api_key.txt") as key_file :
    keepa_api_key = key_file.readline().strip()
    google_api_key = key_file.readline().strip()
api = keepa.Keepa(keepa_api_key)

ConnectionError: HTTPSConnectionPool(host='api.keepa.com', port=443): Max retries exceeded with url: /token/?key=clcar1t67fm5vuqv8rnrum8ot1srjaj62ovru4o8ak577luomjqlr5b09gpckn8f (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fbe662474e0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [3]:
num_searches = 138
search_params = [{}] * num_searches

In [4]:
search_params[0] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 0,
    "current_USED_lte": 4950,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    

}

In [5]:
search_params[1] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 4951,
    "current_USED_lte": 6498,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [6]:
search_params[2] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 6500,
    "current_USED_lte": 6998,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    
    "perPage": 10000
    
}

In [7]:
search_params[3] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 7000,
    "current_USED_lte": 7498,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    
    "perPage": 10000
    
}

In [8]:
search_params[4] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 7500,
    "current_USED_lte": 7998,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
     
    "perPage": 10000
    
}

In [9]:
search_params[5] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 8000,
    "current_USED_lte": 8550,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    
    "perPage": 10000
    
}

In [10]:
search_params[6] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 8551,
    "current_USED_lte": 9050,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [11]:
search_params[7] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 9051,
    "current_USED_lte": 9750,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [12]:
search_params[8] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 9751,
    "current_USED_lte": 10200,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [13]:
search_params[9] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 10201,
    "current_USED_lte": 11190,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [14]:
search_params[10] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 11191,
    "current_USED_lte": 12200,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [15]:
search_params[11] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 12201,
    "current_USED_lte": 13269,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [16]:
search_params[12] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 13271,
    "current_USED_lte": 14800,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [17]:
search_params[13] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 14801,
    "current_USED_lte": 16440,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [18]:
search_params[14] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 16441,
    "current_USED_lte": 18800,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [19]:
search_params[15] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 18801,
    "current_USED_lte": 19999,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [20]:
search_params[16] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 20000,
    "current_USED_lte": 24779,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [21]:
search_params[17] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 24780,
    "current_USED_lte": 32800,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [22]:
search_params[18] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 32801,
    "current_USED_lte": 50000,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [23]:
search_params[19] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 50001,
    "current_USED_lte": 67000,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [24]:
search_params[20] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 67001,
    "current_USED_lte": 89998,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [25]:
search_params[21] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 89999,
    "current_USED_lte": 92300,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [26]:
search_params[22] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 92301,
    "current_USED_lte": 104000,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [27]:
search_params[23] = {
    "avg180_SALES_gte": 6000000,
    "avg180_SALES_lte": 10000000,
    "current_USED_gte": 104001,
    "current_USED_lte": 10000000,
    "avg180_USED_gte": 6500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2500,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 6000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [28]:
search_params[24] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 0,
    "current_USED_lte": 3500,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [29]:
search_params[25] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 3501,
    "current_USED_lte": 4150,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [30]:
search_params[26] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 4151,
    "current_USED_lte": 4540,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [31]:
search_params[27] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 4541,
    "current_USED_lte": 4958,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [32]:
search_params[28] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 4959,
    "current_USED_lte": 5250,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [33]:
search_params[29] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 5251,
    "current_USED_lte": 5700,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [34]:
search_params[30] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 5701,
    "current_USED_lte": 6150,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [35]:
search_params[31] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 6151,
    "current_USED_lte": 6700,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [36]:
search_params[32] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 6701,
    "current_USED_lte": 7298,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [37]:
search_params[33] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 7300,
    "current_USED_lte": 7900,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [38]:
search_params[34] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 7901,
    "current_USED_lte": 8600,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [39]:
search_params[35] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 8601,
    "current_USED_lte": 9450,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [40]:
search_params[36] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 9451,
    "current_USED_lte": 10240,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [41]:
search_params[37] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 10241,
    "current_USED_lte": 11800,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [42]:
search_params[38] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 11801,
    "current_USED_lte": 13600,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [43]:
search_params[39] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 13601,
    "current_USED_lte": 16000,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [44]:
search_params[40] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 16001,
    "current_USED_lte": 19900,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [45]:
search_params[41] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 19901,
    "current_USED_lte": 27000,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [46]:
search_params[42] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 27001,
    "current_USED_lte": 60000,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [47]:
search_params[43] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 60001,
    "current_USED_lte": 90000,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [48]:
search_params[44] = {
    "avg180_SALES_gte": 4000000,
    "avg180_SALES_lte": 5999999,
    "current_USED_gte": 90001,
    "current_USED_lte": 100000000,
    "avg180_USED_gte": 4000,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 2000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 3500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [49]:
search_params[45] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 0,
    "current_USED_lte": 1320,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [50]:
search_params[46] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 1321,
    "current_USED_lte": 1500,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [51]:
search_params[47] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 1501,
    "current_USED_lte": 1595,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [52]:
search_params[48] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 1596,
    "current_USED_lte": 1694,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [53]:
search_params[49] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 1695,
    "current_USED_lte": 1789,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [54]:
search_params[50] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 1791,
    "current_USED_lte": 1889,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [55]:
search_params[51] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 1891,
    "current_USED_lte": 1989,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [56]:
search_params[52] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 1991,
    "current_USED_lte": 2014,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [57]:
search_params[53] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2016,
    "current_USED_lte": 2120,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [58]:
search_params[54] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2121,
    "current_USED_lte": 2245,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [59]:
search_params[55] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2246,
    "current_USED_lte": 2370,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [60]:
search_params[56] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2371,
    "current_USED_lte": 2490,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [61]:
search_params[57] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2491,
    "current_USED_lte": 2550,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [62]:
search_params[58] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2551,
    "current_USED_lte": 2680,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [63]:
search_params[59] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2681,
    "current_USED_lte": 2820,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [64]:
search_params[60] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2821,
    "current_USED_lte": 2970,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [65]:
search_params[61] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 2971,
    "current_USED_lte": 3040,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [66]:
search_params[62] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 3041,
    "current_USED_lte": 3240,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [67]:
search_params[63] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 3240,
    "current_USED_lte": 3400,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [68]:
search_params[64] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 3401,
    "current_USED_lte": 3565,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [69]:
search_params[65] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 3565,
    "current_USED_lte": 3800,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [70]:
search_params[66] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 3801,
    "current_USED_lte": 3997,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [71]:
search_params[67] = {
   "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 3999,
    "current_USED_lte": 4200,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
     
}

In [72]:
search_params[68] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 4201,
    "current_USED_lte": 4480,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [73]:
search_params[69] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 4481,
    "current_USED_lte": 4700,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [74]:
search_params[70] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 4701,
    "current_USED_lte": 4998,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [75]:
search_params[71] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 4999,
    "current_USED_lte": 5350,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [76]:
search_params[72] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 5351,
    "current_USED_lte": 5790,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [77]:
search_params[73] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 5791,
    "current_USED_lte": 6240,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [78]:
search_params[74] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 6241,
    "current_USED_lte": 6900,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [79]:
search_params[75] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 6901,
    "current_USED_lte": 7500,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [80]:
search_params[76] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 7501,
    "current_USED_lte": 8400,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [81]:
search_params[77] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 8401,
    "current_USED_lte": 9400,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [82]:
search_params[78] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 9401,
    "current_USED_lte": 10500,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [83]:
search_params[79] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 10501,
    "current_USED_lte": 12500,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [84]:
search_params[80] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 12501,
    "current_USED_lte": 15500,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [85]:
search_params[81] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 15501,
    "current_USED_lte": 20000,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [86]:
search_params[82] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 20001,
    "current_USED_lte": 40000,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [87]:
search_params[83] = {
    "avg180_SALES_gte": 2400000,
    "avg180_SALES_lte": 3999999,
    "current_USED_gte": 40001,
    "current_USED_lte": 10000000,
    "avg180_USED_gte": 1500,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 1000,
    "avg30_USED_lte": 1000000000,
    "avg90_USED_gte": 1500,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [88]:
search_params[84] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 0,
    "current_USED_lte": 900,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [89]:
search_params[85] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 901,
    "current_USED_lte": 1054,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [90]:
search_params[86] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1055,
    "current_USED_lte": 1149,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [91]:
search_params[87] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1150,
    "current_USED_lte": 1230,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [92]:
search_params[88] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1231,
    "current_USED_lte": 1290,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [93]:
search_params[89] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1291,
    "current_USED_lte": 1340,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [94]:
search_params[90] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1341,
    "current_USED_lte": 1382,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [95]:
search_params[91] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1383,
    "current_USED_lte": 1425,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [96]:
search_params[92] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1426,
    "current_USED_lte": 1482,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [97]:
search_params[93] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1483,
    "current_USED_lte": 1500,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [98]:
search_params[94] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1501,
    "current_USED_lte": 1549,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [99]:
search_params[95] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1550,
    "current_USED_lte": 1598,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [100]:
search_params[96] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1599,
    "current_USED_lte": 1650,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [101]:
search_params[97] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1651,
    "current_USED_lte": 1700,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [102]:
search_params[98] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1701,
    "current_USED_lte": 1770,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [103]:
search_params[99] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1771,
    "current_USED_lte": 1820,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [104]:
search_params[100] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1821,
    "current_USED_lte": 1889,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [105]:
search_params[101] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1891,
    "current_USED_lte": 1948,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [106]:
search_params[102] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1950,
    "current_USED_lte": 1998,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [107]:
search_params[103] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 1998,
    "current_USED_lte": 2020,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [108]:
search_params[104] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2021,
    "current_USED_lte": 2098,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [109]:
search_params[105] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2099,
    "current_USED_lte": 2180,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [110]:
search_params[106] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2181,
    "current_USED_lte": 2250,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [111]:
search_params[107] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2251,
    "current_USED_lte": 2340,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [112]:
search_params[108] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2341,
    "current_USED_lte": 2420,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [113]:
search_params[109] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2421,
    "current_USED_lte": 2498,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [114]:
search_params[110] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2499,
    "current_USED_lte": 2560,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [115]:
search_params[111] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2561,
    "current_USED_lte": 2660,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [116]:
search_params[112] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2661,
    "current_USED_lte": 2780,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [117]:
search_params[113] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2781,
    "current_USED_lte": 2898,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [118]:
search_params[114] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2899,
    "current_USED_lte": 2998,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [119]:
search_params[115] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 2999,
    "current_USED_lte": 3099,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [120]:
search_params[116] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 3100,
    "current_USED_lte": 3250,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [121]:
search_params[117] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 3251,
    "current_USED_lte": 3399,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [122]:
search_params[118] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 3400,
    "current_USED_lte": 3533,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [123]:
search_params[118] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 3534,
    "current_USED_lte": 3700,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [124]:
search_params[119] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 3701,
    "current_USED_lte": 3900,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [125]:
search_params[120] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 3901,
    "current_USED_lte": 4040,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [126]:
search_params[121] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 4040,
    "current_USED_lte": 4250,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [127]:
search_params[122] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 4251,
    "current_USED_lte": 4450,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [128]:
search_params[123] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 4451,
    "current_USED_lte": 4650,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [129]:
search_params[124] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 4451,
    "current_USED_lte": 4650,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [130]:
search_params[125] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 4951,
    "current_USED_lte": 5200,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [131]:
search_params[126] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 5201,
    "current_USED_lte": 5580,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [132]:
search_params[127] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 5581,
    "current_USED_lte": 5999,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [133]:
search_params[128] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 6000,
    "current_USED_lte": 6500,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [134]:
search_params[129] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 6501,
    "current_USED_lte": 7050,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [135]:
search_params[130] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 7051,
    "current_USED_lte": 7700,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [136]:
search_params[131] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 7701,
    "current_USED_lte": 8500,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [137]:
search_params[132] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 8501,
    "current_USED_lte": 9500,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [138]:
search_params[133] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 9501,
    "current_USED_lte": 10800,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [139]:
search_params[134] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 10801,
    "current_USED_lte": 13000,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [140]:
search_params[135] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 13001,
    "current_USED_lte": 17000,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [141]:
search_params[136] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 17001,
    "current_USED_lte": 30000,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [142]:
search_params[137] = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 2399999,
    "current_USED_gte": 30001,
    "current_USED_lte": 100000000,
    "avg180_USED_gte": 1200,
    "avg180_USED_lte": 1000000,
    "rootCategory": 283155,
    "categories_exclude": [
        "10159272011",
        "10399",
        "11280",
        "11357541011",
        "11871",
        "12059",
        "14292116011",
        "18",
        "23",
        "25",
        "2787",
        "282911",
        "2967",
        "2970",
        "2977",
        "3003",
        "3014",
        "301889",
        "3248857011",
        "3284",
        "332930011",
        "3371",
        "4366",
        "4402",
        "4452",
        "4456",
        "4751",
        "6511979011",
        "8944283011"
    ],
    "avg30_USED_gte": 800,
    "avg30_USED_lte": 10000000,
    "avg90_USED_gte": 1000,
    "avg90_USED_lte": 1000000,
    "perPage": 10000
    
}

In [5]:
num_searches = 1

In [6]:
asins = [[] for search in range(num_searches)]
for search in range(num_searches) :
    asins[search] = api.product_finder(search_params[search])

In [7]:
asins = list(itertools.chain.from_iterable(asins))
asins = list(dict.fromkeys(asins))
print('Num asins: ', len(asins))

Num asins:  10000


In [8]:
asins = asins[:100]

In [9]:
def pull_and_analysize(list_of_asins) :
    first_search = api.query(list_of_asins)
    book_data = []
    
    for book in range(len(first_search)) :
        book_data.append(first_search[book])
        try :
            if(first_search[book]['data']['LISTPRICE'][-1] <= 12) :
                book_data.pop()
        except :
            do_nothing = True
        
    print('New num asins: ', len(book_data))
    
    two_yrs_ago = datetime.datetime.now() - relativedelta(years=2)
    sell_prices = pd.DataFrame()
    sell_dates = pd.DataFrame()
    used_counts = pd.DataFrame()
    for book in range(len(book_data)) :
        last_two_years = book_data[book]['data']['SALES_time'] > two_yrs_ago
        drop_dates_list = []
        sell_prices_list = []
        used_counts_list = []
        for day in np.where(last_two_years)[0] :
            if ((book_data[book]['data']['SALES'][day - 1] - 
                book_data[book]['data']['SALES'][day]) / 
                (book_data[book]['data']['SALES'][day - 1]) >= .04) :
                day_sold = book_data[book]['data']['SALES_time'][day]
                drop_dates_list.append(day_sold)
                new = False
                try :
                    day_sold_Used_index = np.where(book_data[book]['data']['USED_time'] == day_sold)[0][0] - 1 
                except IndexError :
                    try :
                        day_sold_Used_index = np.where(book_data[book]['data']['USED_time'] < day_sold)[0][-1]
                    except IndexError :
                        day_sold_Used_index = np.where(book_data[book]['data']['NEW_time'] < day_sold)[0][-1]
                        new = True
                sell_prices_list.append(book_data[book]['data']['NEW' if new else 'USED'][day_sold_Used_index])  
                do_nothing = False
                try :
                    day_sold_used_count = np.where(book_data[book]['data']['COUNT_USED_time'] == day_sold)[0][0] - 1 
                except IndexError :
                    try :
                        day_sold_used_count = np.where(book_data[book]['data']['COUNT_USED_time'] < day_sold)[0][-1]
                    except IndexError :
                        do_nothing = True
                used_counts_list.append(0 if do_nothing else book_data[book]['data']['COUNT_USED'][day_sold_used_count])
        drop_dates_df = pd.DataFrame({book_data[book]['asin']:drop_dates_list})
        sell_dates = pd.concat([sell_dates, drop_dates_df], ignore_index=True, axis=1)
        sell_prices_df = pd.DataFrame({book_data[book]['asin']:sell_prices_list})
        sell_prices = pd.concat([sell_prices, sell_prices_df], ignore_index=True, axis=1)
        used_counts_df = pd.DataFrame({book_data[book]['asin']:used_counts_list})
        used_counts = pd.concat([used_counts, used_counts_df], ignore_index=True, axis=1)

    for column in sell_prices.columns :
        for row in sell_prices.index :
            if(sell_prices[column][row] < 11.77) :
                sell_prices.iloc[row, column] = 0
                used_counts.iloc[row, column] = 0 
                
    sell_prices = sell_prices.fillna(0) #convert the NaNs to 0s so we can do math with them
    used_counts = used_counts.fillna(0)
    sell_dates = sell_dates.fillna(pd.NaT)
    rolling_averages = pd.DataFrame()
    num_sold = pd.DataFrame()
    used_count_avg = pd.DataFrame()
    for book in range(len(book_data)) :
        two_yrs_ago = datetime.datetime.now() - relativedelta(years=2)
        two_yrs_30days = two_yrs_ago + datetime.timedelta(days=30)
        mving_window = [two_yrs_ago, two_yrs_30days]
        rolling_average = []
        books_sold = []
        used_cnt = []
        for day in range(701) : 
            start = sell_dates[book] > mving_window[0]
            end = sell_dates[book] < mving_window[1]
            window = np.where(start & end)[0]
            rolling_average.append(np.sum(sell_prices[book][window]))
            books_sold.append(np.count_nonzero(sell_prices[book][window]))
            used_cnt.append(np.average(used_counts[book][window]) if len(used_counts[book][window]) != 0 else 0)
            for date in range(len(mving_window)) :
                mving_window[date] += datetime.timedelta(days=1)
        rolling_averages[book] = rolling_average
        num_sold[book] = books_sold
        used_count_avg[book] = used_cnt
        
    peaks = pd.DataFrame()
    peak_amount = []
    peak_end_date = []
    peak_num_sold = []
    peak_used_avg = []
    peak_avg_price = []
    now = datetime.datetime.now().date()
    start = now + relativedelta(years=-2,days=30)
    date_range = pd.date_range(start, now)
    for book in range(len(book_data)) :
        peak = np.amax(rolling_averages[book][335:]) # only look at past year
        peak_amount.append(peak)
        peak_time = np.where(rolling_averages[book] == peak)[0]
        peak_end_date.append(date_range[peak_time[-1]])
        peak_num_sold.append(num_sold[book][peak_time[-1]])
        peak_used_avg.append(used_count_avg[book][peak_time[-1]])
        peak_avg_price.append(0 if peak_num_sold[book] == 0 else peak_amount[book] / peak_num_sold[book])
    peaks['Peak Amount'] = peak_amount  
    peaks['Peak End Date'] = peak_end_date
    peaks['Num Sold During Peak'] = peak_num_sold
    peaks['Average # of Used Offers'] = peak_used_avg
    peaks['Average Price'] = peak_avg_price
    
    Amazon_fees = lambda price : price - (price * .15) - 10  
        
    old_peak = pd.DataFrame()
    new_peak_amt = []
    new_peak_date = []
    peak_num_sold = []
    pk_used_avg = []
    pk_avg_price = []
    now = datetime.datetime.now().date()
    start = now + relativedelta(years=-2)
    date_range = pd.date_range(start, now)
    for book in range(len(book_data)) :
        date = peaks['Peak End Date'][book] - relativedelta(years=1) 
        date_index = np.where(date_range == date)[0][0]
        new_peak_amt.append(np.amax(rolling_averages[book][date_index - 5 if date_index >= 5 else 0:date_index + 5]))
        peak_end_index = np.where(rolling_averages[book] == new_peak_amt[book])[0][-1]
        new_peak_date.append(date_range[peak_end_index])
        peak_num_sold.append(num_sold[book][peak_end_index])
        pk_used_avg.append(used_count_avg[book][peak_end_index])
        pk_avg_price.append((new_peak_amt[book] / peak_num_sold[book]) if peak_num_sold[book] != 0 else 0)

    old_peak['Peak Amount'] = new_peak_amt
    old_peak['End Date'] = new_peak_date
    old_peak['Amount Sold'] = peak_num_sold
    old_peak['Avg # of Used Offers'] = pk_used_avg
    old_peak['Average Price'] = pk_avg_price
    
    lambdas = []
    avg_num_sold_peak = []
    discount_factor = 0.8 # the demand for any book this year will descrease by an estimated 20%
    for book in range(len(book_data)) :
        avg_num_sold_peak.append(np.average([peaks['Num Sold During Peak'][book], old_peak['Amount Sold'][book]]))
        lambdas.append(avg_num_sold_peak[book] * discount_factor)

    # given that a book sold, probability that it was our book that sold and not someone else's
    def prob_our_book(used_count) :
        return .9 - (.13 * np.sqrt(used_count))
    
    book_value = []
    roi = []
    asin = []
    peak_avg_price = []
    rankings = pd.DataFrame()
    current_used_count = []
    current_used_price = []
    for book in range(len(book_data)) :
        cur_used = book_data[book]['data']['COUNT_USED'][-1]
        current_used_count.append(0 if cur_used == -1 else cur_used)
        current_used_price.append(book_data[book]['data']['USED'][-1])
        x = 0
        lam = lambdas[book]
        probs = pd.DataFrame()
        p = prob_our_book(current_used_count[book])
        while(stats.poisson.pmf(x, lam) >= .01) :
            dist = stats.binom(x, p)
            prob = []
            for trial in range(x + 1) :
                prob.append(dist.pmf(trial) * stats.poisson.pmf(x, p))
            probs = probs.append(pd.Series(prob), ignore_index=True)
            x += 1
        num_sold_probs = []
        for num in range(len(probs.columns)) :
            num_sold_probs.append(np.sum(probs[num]))
        peak_avg_price.append(np.average([peaks['Average Price'][book], old_peak['Average Price'][book]]))
        book_value.append(Amazon_fees(peak_avg_price[book]) * np.sum(num_sold_probs[1:]))
        roi.append((book_value[book] - add_shipping(current_used_price[book])) / add_shipping(current_used_price[book]))
        asin.append(book_data[book]['asin'])
    rankings['ASIN'] = asin
    rankings['ROI'] = roi
    rankings['Value'] = book_value
    rankings['Peak End Date'] = peaks['Peak End Date']
    rankings['Avg Num Sold During Peak'] = avg_num_sold_peak
    rankings['Current Used Price'] = current_used_price
    rankings = rankings.sort_values(by='ROI', ascending=False)
    
    #now add the appraisal values
    def appraisal_model(used_price, num_sold) :
    if (num_sold == 0) :
        return -1
    else :
        return -.42 + (1/4.5)*used_price + (num_sold - 1)
    
    appraisal_price = []
    for book in range(len(rankings.index)) :
        num_sold = rankings.loc[book, 'Avg Num Sold During Peak']
        used_price = rankings.loc[book, 'Current Used Price']
        appraisal_price.append(appraisal_model(used_price, num_sold))
    rankings['Appraisal Price'] = appraisal_price
    
    #now grab the title 
    api_uri = 'https://www.googleapis.com/books/v1/volumes?q=isbn:'
    key_part = '&key='
    titles = []
    for isbn in rankings['ASIN'] :
        response = requests.get(api_uri + isbn + key_part + google_api_key)
        soup = BeautifulSoup(response.content, 'html.parser')
        google_book_data = json.loads(soup.text.replace('\n', ''))
        title = google_book_data['items'][0]['volumeInfo']['title']
        titles.append(title) 
    rankings['Title'] = titles
    
    return rankings

In [10]:
def add_shipping(price) :
    if price < 5.46 :
        return(price + 3.99)
    elif price < 10 :
        return(price + 2)
    else :
        return price

In [11]:
def pull_and_update(rankings) :
    print('Getting the latest used price data... Today\'s date: ', datetime.datetime.now())
    new_asins = rankings['ASIN'].values
    new_data = api.query(new_asins)
    new_roi = []
    for book in range(len(new_data)) :
        current_used_price = new_data[book]['data']['USED'][-1]
        roi = (rankings['Value'][book] - add_shipping(current_used_price)) / add_shipping(current_used_price)
        new_roi.append(roi)
    new_rankings = rankings.copy()
    new_rankings.loc[:,'ROI'] = new_roi
    new_rankings = new_rankings.sort_values(by='ROI', ascending=False)
    return new_rankings

In [13]:
if __name__ == '__main__' :
    num_processes = 30
    pool = Pool(processes=num_processes)
    pd.set_option('display.max_rows', 500)
    n = round(len(asins) / num_processes)
    split_asins = [asins[i:i+n] for i in range(0, len(asins), n)]
    while(True) :
        last_updated = datetime.datetime.now().date()
        list_of_rankings_df = pool.map(pull_and_analysize, split_asins)
        rankings = pd.concat(list_of_rankings_df, ignore_index=True, axis=0)
        rankings = rankings.sort_values(by='ROI', ascending=False)
        print(rankings)
        rankings.to_csv('results.csv')
        while(datetime.datetime.now().date() - last_updated < datetime.timedelta(30)) :
            top_quarter = round(len(asins) / 4)
            chunk = round(top_quarter / num_processes)
            split_rankings = [rankings.copy().iloc[i:i+chunk] for i in range(0, top_quarter, chunk)]
            for mini_df in split_rankings :
                mini_df.reset_index(inplace=True)
            updated_rankings_list = pool.map(pull_and_update, split_rankings)
            updated_rankings = pd.concat(updated_rankings_list, ignore_index=True, axis=0)
            updated_rankings = updated_rankings.sort_values(by='ROI', ascending=False)
            print(updated_rankings)